# Trips in time and space order
Sorts output trips in time and space order, which is useful for disaggregate (individual) dynamic traffic assignment and person time/space visualization.  Trips in time and space order means the trip origin, destination, and depart period from one trip to the next makes sense.

# Input and output filenames

In [1]:
pipeline_filename = 'example/output/pipeline.h5'
output_trip_filename = "example/output/final_trips_time_space_order.csv"

# Libraries

In [2]:
import pandas as pd
import numpy as np
import itertools

# Read tables directly from the pipeline

In [19]:
# get tables (if run as mp then trip_mode_choice is final state of the tables) 
pipeline = pd.io.pytables.HDFStore(pipeline_filename)
tours = pipeline['/tours/stop_frequency']
trips = pipeline['/trips/trip_mode_choice']
jtp = pipeline['/joint_tour_participants/joint_tour_participation']

# Add related fields, including joint trip participants

In [20]:
trips["tour_participants"] = trips.tour_id.map(tours.number_of_participants)
trips["tour_category"] = trips.tour_id.map(tours.tour_category)
trips["parent_tour_id"] = trips.tour_id.map(tours.index.to_series()).map(tours.parent_tour_id)
trips["tour_start"] = trips.tour_id.map(tours.start)
trips["parent_tour_start"] = trips.parent_tour_id.map(tours.start)
trips["parent_tour_end"] = trips.parent_tour_id.map(tours.end)

# Create additional trips records for other persons on joint trips

In [21]:
tour_person_ids = jtp.groupby("tour_id").apply(lambda x: pd.Series({"person_ids": " ".join(x["person_id"].astype("str"))}))
trips = trips.join(tour_person_ids, "tour_id")
trips["person_ids"] = trips["person_ids"].fillna("")
trips.person_ids = trips.person_ids.where(trips.person_ids!="", trips.person_id)
trips["person_ids"] = trips["person_ids"].astype(str)

person_ids = [*map(lambda x: x.split(" "),trips.person_ids.tolist())]
person_ids = list(itertools.chain.from_iterable(person_ids))

trips_expanded = trips.loc[np.repeat(trips.index, trips['tour_participants'])]
trips_expanded.person_id = person_ids

# Pull out joint and at-work trips 

In [44]:
trips_expanded["inbound"] = ~trips_expanded.outbound
atwork_trips = trips_expanded[trips_expanded.tour_category == "atwork"]
joint_trips = trips_expanded[trips_expanded.tour_category == "joint"]
mnm_trips = trips_expanded[(trips_expanded.tour_category == "mandatory") | (trips_expanded.tour_category == "non_mandatory")]

# Sort records

In [45]:
mnm_trips = mnm_trips.sort_values(['person_id','tour_start','tour_id','inbound','trip_num'])
atwork_trips = atwork_trips.sort_values(['person_id','tour_start','tour_id','inbound','trip_num'])
joint_trips = joint_trips.sort_values(['person_id','tour_start','tour_id','inbound','trip_num'])

# Put at-work trips back in at the correct spot

In [170]:
mnm_trips["trip_id"] = mnm_trips.index
atwork_trips["trip_id"] = atwork_trips.index

mnm_last_trips = mnm_trips[mnm_trips.trip_num == mnm_trips.trip_count]
parent_tour_trips_with_atwork_trips = mnm_last_trips.merge(atwork_trips, left_on="tour_id", right_on="parent_tour_id")
parent_tour_trips_with_atwork_trips["atwork_depart_after"] = parent_tour_trips_with_atwork_trips.eval("depart_y >= depart_x")

parent_trip_id = parent_tour_trips_with_atwork_trips[parent_tour_trips_with_atwork_trips["atwork_depart_after"]][["trip_id_x","trip_id_y"]]
parent_trip_id.index = parent_trip_id["trip_id_y"]
parent_trip_id["parent_trip_id"] = parent_trip_id["trip_id_x"]

parent_trip_id["parent_trip_id"] = parent_trip_id["parent_trip_id"].astype('complex128')

parent_trip_id["parent_trip_id"] = parent_trip_id["parent_trip_id"] + 0.1
parent_trip_id["parent_trip_id"] = parent_trip_id["parent_trip_id"].where(~parent_trip_id["parent_trip_id"].duplicated(), parent_trip_id["parent_trip_id"] + 0.1)
parent_trip_id["parent_trip_id"] = parent_trip_id["parent_trip_id"].where(~parent_trip_id["parent_trip_id"].duplicated(), parent_trip_id["parent_trip_id"] + 0.1)
parent_trip_id["parent_trip_id"] = parent_trip_id["parent_trip_id"].where(~parent_trip_id["parent_trip_id"].duplicated(), parent_trip_id["parent_trip_id"] + 0.1)
parent_trip_id["parent_trip_id"] = parent_trip_id["parent_trip_id"].where(~parent_trip_id["parent_trip_id"].duplicated(), parent_trip_id["parent_trip_id"] + 0.1)

parent_trip_id = parent_trip_id[~parent_trip_id.trip_id_y.duplicated()]

atwork_trips["trip_id"] = atwork_trips["trip_id"].astype('complex128')

atwork_trips["parent_trip_id"] = atwork_trips.trip_id.map(parent_trip_id.parent_trip_id)

atwork_trips["trip_id"] = parent_trip_id["parent_trip_id"]

# Write all trips

In [171]:
mnm_trips["trip_id"] = mnm_trips["trip_id"].astype('complex128')

all_trips = mnm_trips.append(atwork_trips)#.append(joint_trips)

all_trips = all_trips.sort_values(['trip_id'])
all_trips.to_csv(output_trip_filename)